Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [138]:
from random import random, seed
from itertools import product, accumulate
import numpy as np
from tqdm.auto import tqdm
from matplotlib import pyplot as plt

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [139]:
UNIVERSE_SIZE = 10_000
NUM_SETS = 1_000
DENSITY = 0.1

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [140]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [141]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    phenotype = np.logical_or.reduce(SETS[solution])
    return np.all(phenotype)


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

## Have Fun!

In [142]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True)
ic(valid(solution), cost(solution))
None

ic| valid(solution): np.True_
    cost(solution): np.float64(1998336.2152035171)


In [143]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5
ic(valid(solution), cost(solution))
None

ic| valid(solution): np.True_
    cost(solution): np.float64(1047130.1069848168)


# Simple RMHC

In [144]:
def single_mutaion(solution: np.ndarray) -> np.ndarray:
    new_solution = solution.copy()
    index = rng.integers(0, NUM_SETS)
    new_solution[index] = not new_solution[index]
    return new_solution

def fitness(solution: np.ndarray):
    return (valid(solution), -cost(solution))

In [145]:
solution = rng.random(NUM_SETS) < 1
solution_fitness = fitness(solution)
history = [float(solution_fitness[1])]
ic(solution_fitness)

tweak = single_mutaion

for steps in range(10_000):
    new_solution = tweak(solution)
    f = fitness(new_solution)
    history.append(float(f[1]))
    if (f > solution_fitness):
        solution = new_solution
        solution_fitness = fitness(solution)
        #ic(fitness(solution))

ic(fitness(solution))
None

ic| solution_fitness: (np.True_, np.float64(-1998336.2152035171))
ic| fitness(solution): (np.True_, np.float64(-113381.19676734449))


# Multiple RMHC

In [146]:
def multiple_mutation(solution: np.ndarray) -> np.ndarray:
    mask = rng.random(NUM_SETS) < 0.5
    new_solution = np.logical_xor(solution, mask)
    return new_solution

In [147]:
solution = rng.random(NUM_SETS) < 0.3
solution_fitness = fitness(solution)
history = [float(solution_fitness[1])]
ic(solution_fitness)

tweak = multiple_mutation

for steps in range(10_000):
    new_solution = tweak(solution)
    f = fitness(new_solution)
    history.append(float(f[1]))
    if (f > solution_fitness):
        solution = new_solution
        solution_fitness = fitness(solution)
        #ic(fitness(solution))

ic(fitness(solution))
None

ic| solution_fitness: (np.True_, np.float64(-585465.924202941))
ic| fitness(solution): (np.True_, np.float64(-585465.924202941))


# Imporoved RMHC

In [148]:
def multiple_mutation_strength(solution: np.ndarray, strength: float = 0.3) -> np.ndarray:
    mask = rng.random(NUM_SETS) < strength
    if not np.any(mask):
        mask[np.random.randint(NUM_SETS)] = True
    new_solution = np.logical_xor(solution, mask)
    return new_solution

In [149]:
solution = rng.random(NUM_SETS) < 0.8
BUFFER_SIZE = 13
solution_fitness = fitness(solution)
history = [float(solution_fitness[1])]
ic(solution_fitness)

tweak = multiple_mutation_strength

strength = 0.5
buffer = list()

for steps in range(10_000):
    new_solution = tweak(solution, strength)
    f = fitness(new_solution)
    history.append(float(f[1]))
    buffer.append(f > solution_fitness)

    buffer = buffer[-BUFFER_SIZE:]
    if sum(buffer) > BUFFER_SIZE / 2:
        strength *= 1.2
    elif sum(buffer) < BUFFER_SIZE / 2:
        strength /= 1.2

    if (f > solution_fitness):
        solution = new_solution
        solution_fitness = fitness(solution)

ic(fitness(solution))
None

ic| solution_fitness: (np.True_, np.float64(-1586975.2369729434))
ic| fitness(solution): (np.True_, np.float64(-110547.36470757202))
